# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
# imports

import json
import gradio as gr
from AISystem import AISystem, AI, formatPrompt, load_allbots

In [2]:
openai_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-1.5-flash"
ollama_model = "llama3.2"

openai, claude, gemini, ollama = load_allbots()

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always get a price confirmation from the traveler before booking the flight."
system_message += "Always ask the departure city before confirm the booking."
system_message += "Always be accurate. If you don't know the answer, say so."

system_prompt = formatPrompt("system", system_message)
geminiSys = AISystem(gemini,
                        system_prompt, 
                        gemini_model,
                        AI.GEMINI)

openAiSys = AISystem(openai,
                        system_prompt, 
                        openai_model,
                        AI.OPEN_AI)

claudeSys = AISystem(claude,
                       system_prompt, 
                       claude_model,
                        AI.CLAUDE)

ollamaSys = AISystem(ollama,
                        system_prompt, 
                        ollama_model,
                        AI.OLLAMA)
sys_dict = { AI.GEMINI: geminiSys, AI.OPEN_AI: openAiSys, AI.CLAUDE: claudeSys, AI.OLLAMA: ollamaSys}

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyC-


In [3]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [4]:
# There's a particular dictionary structure that's required to describe our function:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [5]:
def book_ticket(destination_city):
    print(f"Tool book_ticket called for {destination_city}")
    return (f"The ticket to {destination_city} has been booked. Thank you.")

In [6]:
book_function = {
    "name": "book_ticket",
    "description": "Get the confirmation of booking a ticket to the destination city. Call this whenever you need to book the ticket, for example when a customer asks 'I'd like to book this ticket'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to book the ticket",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [7]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function},
        {"type": "function", "function": book_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [8]:
  
def chat(message, history, llms):
    aiSystem = sys_dict.get(AI[llms.upper()])
    aiSystem.messages = history
    response = aiSystem.call(formatPrompt("user",message), tools)
    
    if response.finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(messages = messages)
        aiSystem.messages.append(message)
        aiSystem.messages.append(response)
        response = aiSystem.call()
    
    return response.message.content

In [9]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    if price != "Unknown":
        ticket_booked = book_ticket(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,
                               "price": price,
                              "ticket_booked": ticket_booked}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city

In [10]:
llms_dropdown = gr.Dropdown(choices=[ai.value for ai in AI], label="Select model")
chat = gr.ChatInterface(fn=chat, 
                 type="messages",
                 additional_inputs=[llms_dropdown])
chat.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
